<a href="https://colab.research.google.com/github/yasin-sazid/university-schedule-optimizer/blob/main/university_scheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.31.1 which is incompatible.
tensorflow 2.18.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.

In [4]:
from ortools.sat.python import cp_model
import pandas as pd

# Track all penalties to minimize
penalty_vars = []
penalty_weights = []  # Optional: can assign different weights to room vs time

# Load Excel files
classrooms_df = pd.read_excel("data.xlsx", sheet_name="classrooms")
slots_df = pd.read_excel("data.xlsx", sheet_name="slots")
schedule_df = pd.read_excel("sample_schedule.xlsx", sheet_name="Sheet1")

# Step 1: Clean and tag schedule data
valid_schedule = schedule_df[schedule_df['Course'].notna() & schedule_df['Instructor'].notna()].copy()
valid_schedule['CourseID'] = valid_schedule['Course'].astype(str) + "_" + valid_schedule['Section'].astype(str)
valid_schedule['IsLab'] = valid_schedule['Course'].str.contains(r'Lab', case=False, na=False)

valid_schedule['Time From'] = pd.to_datetime(valid_schedule['Time From']).dt.strftime("%H:%M:%S")
valid_schedule['Time To']   = pd.to_datetime(valid_schedule['Time To']).dt.strftime("%H:%M:%S")

# Step 1.5: Expand Weekday column into list of days
def expand_days(day_str):
    return list(day_str.strip()) if isinstance(day_str, str) else []

valid_schedule['Days'] = valid_schedule['Weekday'].apply(expand_days)
valid_schedule['NumDays'] = valid_schedule['Days'].apply(len)

# Load designation sheet (last sheet)
designations_df = pd.read_excel("data.xlsx", sheet_name="faculties")  # last sheet
designations_df.columns = designations_df.columns.str.strip().str.lower()

# Assuming columns are like: 'initial', 'designation'
# Normalize column names
designations_df.rename(columns={'initial': 'Instructor', 'designation': 'Designation'}, inplace=True)

# Merge with valid_schedule
valid_schedule = valid_schedule.merge(designations_df, on='Instructor', how='left')

valid_schedule = valid_schedule.loc[:, ~valid_schedule.columns.str.contains('^Unnamed')]

print(valid_schedule.head(50))

# Room and time slot encoding
room_list = classrooms_df['Room'].dropna().unique().tolist()
room_index = {room: idx for idx, room in enumerate(room_list)}
room_type_map = dict(zip(classrooms_df['Room'], classrooms_df['Theory/Lab']))
room_size_map = dict(zip(classrooms_df['Room'], classrooms_df['Size']))

slots_df['TimeSlot'] = (
    slots_df['Time From'].astype(str) + "-" +
    slots_df['Time To'].astype(str) + "-" +
    slots_df['Weekday']
)

print(slots_df.groupby(['Weekday', 'SlotType']).size())

timeslot_list = slots_df['TimeSlot'].unique().tolist()
timeslot_index = {slot: idx for idx, slot in enumerate(timeslot_list)}
slot_types = slots_df['SlotType'].tolist()

print("\n🧾 Course Summary:")
for _, row in valid_schedule.iterrows():
    print(f"• {row['CourseID']}: Days={row['Days']}, IsLab={row['IsLab']}, Size={row['Size']}, Instructor={row['Instructor']}")

# Step 2: Create variables
model = cp_model.CpModel()
room_vars = {}
timeslot_vars = {}

for _, row in valid_schedule.iterrows():
    cid = row['CourseID']
    days = row['Days']

    room_vars[cid] = model.NewIntVar(0, len(room_list) - 1, f"room_{cid}")
    timeslot_vars[cid] = [
        model.NewIntVar(0, len(timeslot_list) - 1, f"time_{cid}_{d}")
        for d in days
    ]

# Step 3: Time slot filtering based on type and day
from datetime import datetime

# Map timeslot index to its day and slot type
slot_info = {
    idx: {
        'day': ts.split('-')[-1],
        'slot_type': slots_df.iloc[idx]['SlotType']
    }
    for idx, ts in enumerate(timeslot_list)
}

# Step 3: Timeslot must match correct day and slot type

from datetime import datetime, timedelta

def parse_time_duration(t_from, t_to):
    try:
        t1 = pd.to_datetime(t_from).time()
        t2 = pd.to_datetime(t_to).time()

        start = datetime.combine(datetime.today(), t1)
        end = datetime.combine(datetime.today(), t2)

        # If end is earlier, it's probably PM (Excel time logic)
        if end <= start:
            end = datetime.combine(datetime.today(), t2) + timedelta(hours=12)

        duration = (end - start).seconds / 3600
        return duration
    except Exception as e:
        print(f"❌ Failed to parse duration from {t_from} to {t_to}: {e}")
        return 0

from datetime import datetime

slot_info = {
    idx: {
        'day': ts.split('-')[-1],
        'slot_type': slots_df.iloc[idx]['SlotType']
    }
    for idx, ts in enumerate(timeslot_list)
}

for _, row in valid_schedule.iterrows():
    cid = row['CourseID']
    days = row['Days']
    is_lab = row['IsLab']
    time_from = str(row['Time From']).strip()
    time_to = str(row['Time To']).strip()

    try:
        duration_hours = parse_time_duration(time_from, time_to)
    except:
        duration_hours = 0

    for i, d in enumerate(days):
        allowed_types = []

        if is_lab:
            if 2.8 <= duration_hours <= 3.2:
                allowed_types = ['Lab-3hr']
            elif 1.7 <= duration_hours <= 2.3:
                allowed_types = ['Lab-2hr']
            elif 1.3 <= duration_hours <= 1.7:
                allowed_types = ['Theory']
        else:
            allowed_types = ['Theory']

        valid_indices = [
            idx for idx, info in slot_info.items()
            if info['day'] == d and info['slot_type'] in allowed_types
        ]

        if not valid_indices:
            print(f"⚠️ Fallback: No matching slot for {cid} on {d} (duration={duration_hours:.2f}h), relaxing...")
            valid_indices = [
                idx for idx, info in slot_info.items()
                if info['day'] == d
            ]

        model.AddAllowedAssignments([timeslot_vars[cid][i]], [[j] for j in valid_indices])

#Step 3.5: Preference by Designation
for _, row in valid_schedule.iterrows():
    designation = str(row.get('Designation', '')).lower()
    if designation.strip().lower() in ['professor', 'dean', 'head']:
        cid = row['CourseID']
        preferred_room = row['Room']
        time_from = row['Time From']
        time_to = row['Time To']
        days = row['Days']

        # Room match penalty (strong soft constraint)
        if preferred_room in room_index:
            room_match = model.NewBoolVar(f"room_forced_{cid}")
            model.Add(room_vars[cid] == room_index[preferred_room]).OnlyEnforceIf(room_match)
            model.Add(room_vars[cid] != room_index[preferred_room]).OnlyEnforceIf(room_match.Not())
            # penalty_vars.append(room_match.Not())
            # penalty_weights.append(100)  # high penalty
            penalty_vars.append((room_match.Not(), f"Room fixed for {cid} (Prof/Dean/Head)"))
            penalty_weights.append(100)

        # Timeslot match penalties (per day)
        for i, d in enumerate(days):
            matching_slot = f"{time_from}-{time_to}-{d}"
            if matching_slot in timeslot_index:
                t_match = model.NewBoolVar(f"timeslot_forced_{cid}_{d}")
                model.Add(timeslot_vars[cid][i] == timeslot_index[matching_slot]).OnlyEnforceIf(t_match)
                model.Add(timeslot_vars[cid][i] != timeslot_index[matching_slot]).OnlyEnforceIf(t_match.Not())
                # penalty_vars.append(t_match.Not())
                # penalty_weights.append(100)  # strong soft constraint
                penalty_vars.append((t_match.Not(), f"Time fixed for {cid} (Prof/Dean/Head)"))
                penalty_weights.append(100)
            else:
                print(f"⚠️ Slot {matching_slot} not found for {cid}")

    # Preference by Adjunct Faculty
    if 'adjunct' in designation:
        cid = row['CourseID']
        time_from = row['Time From']
        time_to = row['Time To']
        days = row['Days']

        # Timeslot match penalties (per day)
        for i, d in enumerate(days):
            matching_slot = f"{time_from}-{time_to}-{d}"
            if matching_slot in timeslot_index:
                t_match = model.NewBoolVar(f"adjunct_time_pref_{cid}_{d}")
                model.Add(timeslot_vars[cid][i] == timeslot_index[matching_slot]).OnlyEnforceIf(t_match)
                model.Add(timeslot_vars[cid][i] != timeslot_index[matching_slot]).OnlyEnforceIf(t_match.Not())
                # penalty_vars.append(t_match.Not())
                # penalty_weights.append(100)  # medium-high weight for adjunct faculty
                penalty_vars.append((t_match.Not(), f"Time preference for Adjunct {cid}"))
                penalty_weights.append(100)
            else:
                print(f"⚠️ Slot {matching_slot} not found for Adjunct {cid}")

    # Time preference for remaining faculty members
    is_special = any(key in designation for key in ['professor', 'dean', 'head', 'adjunct'])

    if not is_special:
        cid = row['CourseID']
        time_from = row['Time From']
        time_to = row['Time To']
        days = row['Days']

        for i, d in enumerate(days):
            matching_slot = f"{time_from}-{time_to}-{d}"
            if matching_slot in timeslot_index:
                t_match = model.NewBoolVar(f"pref_time_basic_{cid}_{d}")
                model.Add(timeslot_vars[cid][i] == timeslot_index[matching_slot]).OnlyEnforceIf(t_match)
                model.Add(timeslot_vars[cid][i] != timeslot_index[matching_slot]).OnlyEnforceIf(t_match.Not())

                # Assign low weight (e.g., 20) for basic time preference
                penalty_vars.append((t_match.Not(), f"Basic time preference mismatch for {cid}"))
                penalty_weights.append(20)
            else:
                print(f"⚠️ No exact preferred time match for general faculty {cid} on {d}")

# Step 3.5.3: Main Building preference for Senior Faculties
for _, row in valid_schedule.iterrows():
    cid = row['CourseID']
    designation = str(row.get('Designation', '')).lower()

    if any(key in designation for key in ['professor', 'associate', 'adjunct']):
        # Preferred building is "Main"
        preferred_building = "Main"
        preferred_rooms = [
            r for r in room_list
            if str(classrooms_df[classrooms_df['Room'] == r]['Building'].values[0]).strip().lower() == preferred_building.lower()
        ]

        if preferred_rooms:
            valid_room_indices = [room_index[r] for r in preferred_rooms]
            building_match = model.NewBoolVar(f"building_pref_{cid}")
            model.AddAllowedAssignments([room_vars[cid]], [[i] for i in valid_room_indices]).OnlyEnforceIf(building_match)
            model.AddForbiddenAssignments([room_vars[cid]], [[i] for i in valid_room_indices]).OnlyEnforceIf(building_match.Not())
            # penalty_vars.append(building_match.Not())
            # penalty_weights.append(80)  # Soft penalty weight between #1 (100) and #2 (90)
            penalty_vars.append((building_match.Not(), f"Main Building preference for {cid}"))
            penalty_weights.append(80)

# Step 3.5.4: Building preference constraint (FUB/AB1/AB3) for certain faculty offices
# Uses: room_vars, room_index, room_building_map, valid_schedule['Building']
# Normalize building preference
preferred_buildings = ['FUB', 'AB1', 'AB3']
room_building_map = dict(zip(classrooms_df['Room'], classrooms_df['Building']))

# Map room → index
for _, row in valid_schedule.iterrows():
    cid = row['CourseID']
    instructor_building = str(row.get('Building', '')).strip().upper()

    if instructor_building in preferred_buildings:
        # Filter rooms in same building
        preferred_room_indices = [
            idx for room, idx in room_index.items()
            if room_building_map.get(room, '').strip().upper() == instructor_building
        ]

        if preferred_room_indices:
            # Add soft penalty for violating preferred building
            room_match = model.NewBoolVar(f"room_building_match_{cid}")
            model.AddAllowedAssignments([room_vars[cid]], [[r] for r in preferred_room_indices]).OnlyEnforceIf(room_match)
            model.AddForbiddenAssignments([room_vars[cid]], [[r] for r in preferred_room_indices]).OnlyEnforceIf(room_match.Not())
            # penalty_vars.append(room_match.Not())  # 1 if not matched
            # penalty_weights.append(60)  # Medium priority soft constraint
            penalty_vars.append((room_match.Not(), f"Faculty office building preference for {cid}"))
            penalty_weights.append(60)

# Step 3.5.5: Back-to-back classes → Prefer same building for faculty
from datetime import datetime

print("\n🏃 Back-to-back same building constraint:")
time_format = "%H:%M:%S"

# Parse timeslot start-end for each index
timeslot_time_map = {}
for idx, slot in enumerate(timeslot_list):
    start, end, day = slot.split("-")
    timeslot_time_map[idx] = {
        "day": day,
        "start": datetime.strptime(start, time_format),
        "end": datetime.strptime(end, time_format)
    }

# Group course IDs by instructor
instructor_courses = valid_schedule.groupby("Instructor")["CourseID"].apply(list).to_dict()

for instructor, cids in instructor_courses.items():
    for d in ['S', 'M', 'T', 'W', 'R']:
        # Gather all (cid, day_index, slot_var) tuples for this instructor on day d
        day_slots = []
        for cid in cids:
            row = valid_schedule[valid_schedule["CourseID"] == cid]
            if row.empty:
                continue
            days = row.iloc[0]['Days']
            for i, day_code in enumerate(days):
                if day_code == d:
                    day_slots.append((cid, i, timeslot_vars[cid][i]))

        # Compare each pair for back-to-back same-day classes
        for i in range(len(day_slots)):
            cid1, i1, slot_var1 = day_slots[i]
            for j in range(i + 1, len(day_slots)):
                cid2, i2, slot_var2 = day_slots[j]

                for idx1 in range(len(timeslot_list)):
                    info1 = timeslot_time_map[idx1]
                    if info1["day"] != d:
                        continue
                    for idx2 in range(len(timeslot_list)):
                        info2 = timeslot_time_map[idx2]
                        if info2["day"] != d:
                            continue
                        # Check if classes are back-to-back (<=10 min gap)
                        gap1 = (info2["start"] - info1["end"]).total_seconds() / 60
                        gap2 = (info1["start"] - info2["end"]).total_seconds() / 60
                        if 0 <= gap1 <= 10 or 0 <= gap2 <= 10:
                            # If both slots assigned, enforce same building
                            cond1 = model.NewBoolVar(f"cid1_match_{cid1}_{cid2}_{d}")
                            cond2 = model.NewBoolVar(f"cid2_match_{cid1}_{cid2}_{d}")
                            both = model.NewBoolVar(f"both_{cid1}_{cid2}_{d}")

                            model.Add(slot_var1 == idx1).OnlyEnforceIf(cond1)
                            model.Add(slot_var1 != idx1).OnlyEnforceIf(cond1.Not())
                            model.Add(slot_var2 == idx2).OnlyEnforceIf(cond2)
                            model.Add(slot_var2 != idx2).OnlyEnforceIf(cond2.Not())
                            model.AddBoolAnd([cond1, cond2]).OnlyEnforceIf(both)
                            model.AddBoolOr([cond1.Not(), cond2.Not()]).OnlyEnforceIf(both.Not())

                            # Violation if buildings not equal
                            building_mismatch = model.NewBoolVar(f"building_mismatch_{cid1}_{cid2}_{d}")
                            model.Add(room_vars[cid1] != room_vars[cid2]).OnlyEnforceIf(building_mismatch)
                            model.Add(room_vars[cid1] == room_vars[cid2]).OnlyEnforceIf(building_mismatch.Not())

                            # Penalty if both are back-to-back AND building mismatch
                            violation = model.NewBoolVar(f"bb_violation_{cid1}_{cid2}_{d}")
                            model.AddBoolAnd([both, building_mismatch]).OnlyEnforceIf(violation)
                            model.AddBoolOr([both.Not(), building_mismatch.Not()]).OnlyEnforceIf(violation.Not())

                            # penalty_vars.append(violation)
                            # penalty_weights.append(70)
                            penalty_vars.append((violation, f"Back-to-back building mismatch: {cid1} & {cid2} on {d}"))
                            penalty_weights.append(70)

# Step 3.5.6: Idle Time Constraints
print("\n🕒 Idle Time Constraints:")

from datetime import datetime

time_format = "%H:%M:%S"
weekly_idle = {}

for instructor, cids in instructor_courses.items():
    for d in ['S', 'M', 'T', 'W', 'R']:
        slot_vars = []
        for cid in cids:
            row = valid_schedule[valid_schedule["CourseID"] == cid]
            if row.empty:
                continue
            days = row.iloc[0]['Days']
            for i, day_code in enumerate(days):
                if day_code == d:
                    slot_vars.append(timeslot_vars[cid][i])

        if len(slot_vars) < 2:
            continue  # No idle time possible

        earliest = model.NewIntVar(0, len(timeslot_list) - 1, f"{instructor}_{d}_first")
        latest   = model.NewIntVar(0, len(timeslot_list) - 1, f"{instructor}_{d}_last")

        model.AddMinEquality(earliest, slot_vars)
        model.AddMaxEquality(latest, slot_vars)

        # Get start/end minutes for each slot
        start_minutes = [int(datetime.strptime(slot.split("-")[0], time_format).hour) * 60 +
                         int(datetime.strptime(slot.split("-")[0], time_format).minute)
                         for slot in timeslot_list]
        end_minutes = [int(datetime.strptime(slot.split("-")[1], time_format).hour) * 60 +
                       int(datetime.strptime(slot.split("-")[1], time_format).minute)
                       for slot in timeslot_list]

        max_min = max(end_minutes) - min(start_minutes)

        start_lookup = model.NewIntVar(0, max_min, f"{instructor}_{d}_start_min")
        end_lookup   = model.NewIntVar(0, max_min, f"{instructor}_{d}_end_max")

        model.AddElement(earliest, start_minutes, start_lookup)
        model.AddElement(latest, end_minutes, end_lookup)

        duration = model.NewIntVar(0, max_min, f"{instructor}_{d}_span")
        model.Add(duration == end_lookup - start_lookup)

        occupied = model.NewIntVar(0, len(slot_vars) * 180, f"{instructor}_{d}_total_class_time")
        model.Add(occupied == len(slot_vars) * 90)  # each slot is 90 mins

        idle_time = model.NewIntVar(0, max_min, f"{instructor}_{d}_idle")
        model.Add(idle_time == duration - occupied)

        weekly_idle.setdefault(instructor, []).append(idle_time)

        # Soft constraint: daily idle time > 240 min
        idle_violation = model.NewBoolVar(f"{instructor}_{d}_idle_violation")
        model.Add(idle_time > 240).OnlyEnforceIf(idle_violation)
        model.Add(idle_time <= 240).OnlyEnforceIf(idle_violation.Not())

        penalty_vars.append((idle_violation, f"Daily idle >4hr: {instructor} on {d}"))
        penalty_weights.append(50)

# Weekly idle constraint: sum of daily idle time > 660 min (11 hours)
for instructor, idle_vars in weekly_idle.items():
    weekly_total = model.NewIntVar(0, 3000, f"{instructor}_weekly_idle")
    model.Add(weekly_total == sum(idle_vars))

    violation = model.NewBoolVar(f"{instructor}_weekly_idle_violation")
    model.Add(weekly_total > 660).OnlyEnforceIf(violation)
    model.Add(weekly_total <= 660).OnlyEnforceIf(violation.Not())

    penalty_vars.append((violation, f"Weekly idle >11hr: {instructor}"))
    penalty_weights.append(90)

# Step 4: Room-time clash prevention
course_ids = list(valid_schedule['CourseID'])

for i in range(len(course_ids)):
    for j in range(i + 1, len(course_ids)):
        cid1, cid2 = course_ids[i], course_ids[j]

        for idx_a, t1 in enumerate(timeslot_vars[cid1]):
            for idx_b, t2 in enumerate(timeslot_vars[cid2]):
                day_a = valid_schedule.loc[valid_schedule['CourseID'] == cid1, 'Days'].values[0][idx_a]
                day_b = valid_schedule.loc[valid_schedule['CourseID'] == cid2, 'Days'].values[0][idx_b]

                if day_a == day_b:
                    same_room = model.NewBoolVar(f"{cid1}_{cid2}_same_room_{day_a}")
                    same_time = model.NewBoolVar(f"{cid1}_{cid2}_same_time_{day_a}")

                    model.Add(room_vars[cid1] == room_vars[cid2]).OnlyEnforceIf(same_room)
                    model.Add(room_vars[cid1] != room_vars[cid2]).OnlyEnforceIf(same_room.Not())

                    model.Add(t1 == t2).OnlyEnforceIf(same_time)
                    model.Add(t1 != t2).OnlyEnforceIf(same_time.Not())

                    model.AddBoolOr([same_room.Not(), same_time.Not()])

# Step 5: Instructor conflict
instructor_courses = valid_schedule.groupby('Instructor')['CourseID'].apply(list).to_dict()
print("\n👨‍🏫 Instructor Course Mapping:")
for instructor, cids in instructor_courses.items():
    print(f"• {instructor} teaches {len(cids)} courses → {cids}")
    if len(cids) > 5:
        print(f"⚠️ Instructor {instructor} has high teaching load")
    for i in range(len(cids)):
        for j in range(i + 1, len(cids)):
            cid1, cid2 = cids[i], cids[j]
            for t1 in timeslot_vars[cid1]:
                for t2 in timeslot_vars[cid2]:
                    model.Add(t1 != t2)

# Step 6: Room capacity
print("\n🏫 Valid Room Mapping for Capacity:")
for _, row in valid_schedule.iterrows():
    cid = row['CourseID']
    size = int(row['Size'])
    valid_room_indices = [
        room_index[room_name]
        for room_name, capacity in room_size_map.items()
        if pd.notna(capacity) and int(capacity) >= size
    ]
    print(f"• {cid} needs size {size} → {len(valid_room_indices)} valid rooms")
    if not valid_room_indices:
        print(f"❌ No rooms with sufficient capacity for {cid}")
    model.AddAllowedAssignments([room_vars[cid]], [[r] for r in valid_room_indices])

# Step 7: Lab room constraint
print("\n🔬 Lab Room Mapping:")
for _, row in valid_schedule.iterrows():
    cid = row['CourseID']
    if row['IsLab']:
        valid_lab_indices = [
            room_index[r] for r in room_type_map
            if isinstance(room_type_map[r], str) and 'lab' in room_type_map[r].lower()
        ]
        print(f"• {cid} is a lab → {len(valid_lab_indices)} candidate rooms")
        if not valid_lab_indices:
            print(f"❌ No lab rooms found for {cid}")
        model.AddAllowedAssignments([room_vars[cid]], [[r] for r in valid_lab_indices])

        from datetime import datetime

def get_duration_hours(row):
    try:
        fmt = "%H:%M"
        start_dt = datetime.strptime(str(row['Time From']).strip(), fmt)
        end_dt = datetime.strptime(str(row['Time To']).strip(), fmt)
        return (end_dt - start_dt).seconds / 3600
    except:
        return 0

valid_schedule['DurationHrs'] = valid_schedule.apply(get_duration_hours, axis=1)

# For lab courses
lab_durations = valid_schedule[valid_schedule['IsLab']]

print("Lab duration breakdown:")
print(lab_durations['DurationHrs'].value_counts(bins=[0, 1.7, 2.3, 3.2], sort=False))

for _, row in valid_schedule.iterrows():
    cid = row['CourseID']
    preferred_room = row['Room'] if pd.notna(row['Room']) else None
    preferred_time_from = row['Time From'] if pd.notna(row['Time From']) else None
    preferred_time_to = row['Time To'] if pd.notna(row['Time To']) else None
    days = row['Days']

    # print(f"⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️")

    # 🏫 ROOM PREFERENCE PENALTY
    if preferred_room in room_index:
        pref_room_idx = room_index[preferred_room]
        room_match = model.NewBoolVar(f"room_pref_match_{cid}")
        model.Add(room_vars[cid] == pref_room_idx).OnlyEnforceIf(room_match)
        model.Add(room_vars[cid] != pref_room_idx).OnlyEnforceIf(room_match.Not())
        # penalty_vars.append(room_match.Not())  # 1 if not matched
        # penalty_weights.append(50)  # e.g., room preference penalty weight
        penalty_vars.append((room_match.Not(), f"Room preference mismatch for {cid}"))
        penalty_weights.append(50)

    # 🕒 TIME PREFERENCE PENALTY (match any day slot)
    try:
        from_str = pd.to_datetime(preferred_time_from).strftime("%H:%M:%S")
        to_str = pd.to_datetime(preferred_time_to).strftime("%H:%M:%S")
        time_str = f"{from_str}-{to_str}"

        matched_indices = [
            idx for idx, slot in enumerate(timeslot_list)
            if slot.startswith(time_str) and slot.split("-")[-1] in days
        ]

        if not matched_indices:
            print(f"❌ No matching slot found for {cid}: preferred {time_str}, days {days}")
        else:
            print(f"✅ {cid} → Preferred {time_str} matched to slots {[timeslot_list[i] for i in matched_indices]}")


        if matched_indices:
            for i, t_var in enumerate(timeslot_vars[cid]):
                time_match = model.NewBoolVar(f"time_pref_match_{cid}_{i}")
                model.AddAllowedAssignments([t_var], [[idx] for idx in matched_indices]).OnlyEnforceIf(time_match)
                model.AddForbiddenAssignments([t_var], [[idx] for idx in matched_indices]).OnlyEnforceIf(time_match.Not())
                # penalty_vars.append(time_match.Not())  # 1 if not matched
                # penalty_weights.append(30)  # e.g., time preference penalty weight
                # penalty_vars.append((time_match.Not(), f"Time preference mismatch for {cid}"))
                # penalty_weights.append(30)
    except:
        pass

# 🧠 MINIMIZE TOTAL PENALTIES
# model.Minimize(sum(w * v for w, v in zip(penalty_weights, penalty_vars)))
model.Minimize(sum(w * var for (var, _), w in zip(penalty_vars, penalty_weights)))


# Step 8: Solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

print(f"\n📊 Solver status: {solver.StatusName(status)}")
print(f"🕒 Wall time: {solver.WallTime():.2f}s")
print(f"🔁 Conflicts: {solver.NumConflicts()}")
print(f"↪️ Branches: {solver.NumBranches()}")

# Only one solution attempted in basic solve, SolutionCount is not available.
print(f"📏 (Note: Only one solution attempted — use a solution callback to enumerate more)")

if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    records = []
    for _, row in valid_schedule.iterrows():
        cid = row['CourseID']
        section = row['Section']
        course = row['Course']
        actual_room = room_list[solver.Value(room_vars[cid])]
        preferred_room = row['Room'] if pd.notna(row['Room']) else None

        for i, d in enumerate(row['Days']):
            ts_index = solver.Value(timeslot_vars[cid][i])
            ts_string = timeslot_list[ts_index]
            time_from, time_to, day = ts_string.split('-')

            pref_time_from = pd.to_datetime(row['Time From']).strftime("%H:%M:%S")
            pref_time_to   = pd.to_datetime(row['Time To']).strftime("%H:%M:%S")

            pref_time_match = (
                pref_time_from == time_from and
                pref_time_to == time_to
            )

            pref_room_match = (
                preferred_room == actual_room
            )


            if not pref_time_match:
                print(f"❌ {cid} did NOT get preferred slot → Time: {row['Time From']}–{row['Time To']} ≠ {time_from}–{time_to}")
            else:
                print(f"✅ {cid} got preferred slot → Time: {row['Time From']}–{row['Time To']} = {time_from}–{time_to}")

            if preferred_room is None:
                print(f"ℹ️ {cid} had no preferred room → Assigned: {actual_room}")
            elif not pref_room_match:
                print(f"❌ {cid} did NOT get preferred room → {preferred_room} ≠ {actual_room}")
            else:
                print(f"✅ {cid} got preferred room → {preferred_room} = {actual_room}")


            # 📝 Final schedule entry
            records.append({
                'Course': course,
                'Section': section,
                'Instructor': row['Instructor'],
                'Day': d,
                'Room': actual_room,
                'Time From': time_from,
                'Time To': time_to
            })

    result_df = pd.DataFrame(records)
    print("🧠 Total courses scheduled:", len(result_df['Course'].unique()))
    print("📚 Total slots used:", len(result_df))
    print("🏫 Room usage breakdown:\n", result_df['Room'].value_counts())
    result_df.to_excel("final_schedule_output.xlsx", index=False)
    print("✅ Output written to final_schedule_output.xlsx")
else:
    print("❌ No feasible solution found.")
    # with open("model_dump.txt", "w") as f:
    #     f.write(str(model))
    # print("📄 Model structure dumped to model_dump.txt")
print("\n📋 Soft Constraint Violations:")
violated_count = 0
for (var, description), weight in zip(penalty_vars, penalty_weights):
    if solver.BooleanValue(var):
        print(f"❌ {description} | Penalty: {weight}")
        violated_count += 1
print(f"🔎 Total Violations: {violated_count} out of {len(penalty_vars)}")


/tmp/ipython-input-259687487.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  valid_schedule['Time From'] = pd.to_datetime(valid_schedule['Time From']).dt.strftime("%H:%M:%S")
/tmp/ipython-input-259687487.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  valid_schedule['Time To']   = pd.to_datetime(valid_schedule['Time To']).dt.strftime("%H:%M:%S")


    Course  Section  Term  Year  Size Time From   Time To Weekday Instructor  \
0   ACT101        1     2  2025    30  10:10:00  11:40:00      MW       MRDI   
1   ACT101        2     2  2025    30  01:30:00  03:00:00      TR        NHN   
2   ACT101        3     2  2025    30  11:50:00  01:20:00      ST        NHN   
3   ACT101        4     2  2025    30  08:30:00  10:00:00      SR        THD   
4   ACT101        5     2  2025    30  11:50:00  01:20:00      ST        THD   
5   ACT101        6     2  2025    30  10:10:00  11:40:00      MW        THD   
6   ACT101        7     2  2025    30  08:30:00  10:00:00      MW       MSQI   
7   ACT101        8     2  2025    30  11:50:00  01:20:00      ST       MSQI   
8   ACT101        9     2  2025    30  01:30:00  03:00:00      MW       MHRN   
9   ACT101       10     2  2025    30  01:30:00  03:00:00      SR       MHRN   
10  ACT101       11     2  2025    30  04:50:00  06:20:00      TR       MHRN   
11  ACT101       12     2  2025    30  1